In [1]:
from opentree import OT

In [ ]:
## Labels from tree at https://github.com/n8upham/MamPhy_v1/blob/master/_DATA|/MamPhy_fullPosterior_BDvr_DNAonly_4098sp_topoFree_NDexp_MCC_v2_target.tre

In [28]:
mammal_spps =  open("mammal/mammal_tre_names.txt").readlines()

In [4]:
#spp_names_ids = dict()
for label in mammal_spps:
    spp = " ".join(label.strip().split("_")[:2])
    if spp not in spp_names_ids:
        match = OT.tnrs_match([spp], do_approximate_matching=False)
        spp_names_ids[spp] = match.response_dict

In [5]:
len(spp_names_ids)

4100

In [6]:
## Example TNRS result

In [8]:
spp_names_ids['Elephantulus brachyrhynchus']

{'context': 'Mammals',
 'governing_code': 'ICZN',
 'includes_approximate_matches': False,
 'includes_deprecated_taxa': False,
 'includes_suppressed_names': False,
 'matched_names': ['Elephantulus brachyrhynchus'],
 'results': [{'matches': [{'is_approximate_match': False,
     'is_synonym': False,
     'matched_name': 'Elephantulus brachyrhynchus',
     'nomenclature_code': 'ICZN',
     'score': 1.0,
     'search_string': 'elephantulus brachyrhynchus',
     'taxon': {'flags': [],
      'is_suppressed': False,
      'is_suppressed_from_synth': False,
      'name': 'Elephantulus brachyrhynchus',
      'ott_id': 741348,
      'rank': 'species',
      'source': 'ott3.2draft9',
      'synonyms': ['Macroscelides brachyrhynchus',
       'Macroscelides brachyrhynchus malosae'],
      'tax_sources': ['ncbi:113275', 'gbif:2440856', 'irmng:10220902'],
      'unique_name': 'Elephantulus brachyrhynchus'}}],
   'name': 'Elephantulus brachyrhynchus'}],
 'taxonomy': {'author': 'open tree of life projec

In [37]:
no_match = set()
fi = open("Mammal_TNRS.tsv", "w")
fi.write("SppName\tLabel\tOttId\tGbifId\tncbiId\tSynonyms\n")
for label in mammal_spps[1:]:
    ott_id = gbif_id = ncbi_id = synonyms = "-"
    spp = " ".join(label.strip().split("_")[:2])
    matches = spp_names_ids[spp]['results'][0]['matches']
    if matches:
        match = matches[0]['taxon']
        ott_id = match['ott_id']
        gbif_id = ""
        ncbi_id = ""
        for source in match['tax_sources']:
            if source.startswith("gbif"):
                gbif_id = gbif_id + source
            if source.startswith("ncbi"):
                ncbi_id = ncbi_id + source
        synonyms = ",".join(match['synonyms'])
    else:
        no_match.add(spp)
    fi.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(spp, label.strip(), ott_id, gbif_id, ncbi_id, synonyms))
fi.close()
    
    

In [38]:
no_match

{'Blanfordimys juldaschi',
 'Crocidura abscondita',
 'Dactylonax palpator',
 'Dermanura aztecus',
 'Dermanura cinereus',
 'Dermanura glaucus',
 'Dermanura gnomus',
 'Dermanura incomitatus',
 'Dermanura rosenbergii',
 'Dermanura toltecus',
 'Eptesicus pachyomus',
 'Gerbilliscus gambiana',
 'Herpestes flavescens',
 'Hipposideros pendelburyi',
 'Homo denisova',
 'Monodelphis umbristriatus',
 'Murina loreliae',
 'Nesotragus moschatus',
 'Nilgiritragus hylocrius',
 'Ningaui yvonneae',
 'Nyctinomops kalinowskii',
 'Otaria bryonia',
 'Petaurista grandis',
 'Petaurista lena',
 'Petaurista marica',
 'Phataginus tetradactyla',
 'Pipistrellus alaschanicus',
 'Platyrrhinus angustirostris',
 'Platyrrhinus fusciventris',
 'Salanoia durrelli',
 'Smutsia gigantea',
 'Soricomys leonardcoi',
 'Tamiops macclellandii'}

In [35]:
fi = open("unmatched_mammal_names.txt","w")
for name in no_match:
    fi.write(name+"\n")
fi.close()

In [36]:
for spp in no_match:
        match = OT.tnrs_match([spp], do_approximate_matching=False)
        spp_names_ids[spp] = match.response_dict

In [ ]:
## To interactively match these names, upload at https://tree.opentreeoflife.org/curator/tnrs/

In [39]:
len(no_match)

33

In [91]:
#Compare our ncbi matches to Nate's file
fi = open("mammal/Mammal_TNRS.tsv")
header= fi.readline()
label_to_ncbi = dict()
spp_set = set()
for lin in fi:
    spp, label, ott_id, gbif_id, ncbi_id, synonyms = lin.split('\t')
    label_to_ncbi[label] = ncbi_id
    spp_set.add(spp)
    for name in synonyms.split(','):
        spp_set.add(name)

    
    

In [69]:
natelinkfi = open("mammal/taxonomy_mamPhy_5911species_toPublish.csv")
taxon_name_to_label = dict()
header = natelinkfi.readline()
for lin in natelinkfi:
    if len(lin) > 3:
        taxon_name = lin.split(',')[0]
        label = lin.split(',')[1]
        taxon_name_to_label[taxon_name] = label
        if label not in label_to_ncbi:
            assert(label not in mammal_spps)

In [78]:
#Compare our ncbi matches to Nate's file
natefi = open("mammal/UphamEtAl_MamPhy_IUCN-to-NCBI_matchup_Sept2015.csv")
natefi.readline()
for lin in natefi:
    if len(lin) > 10:
        spp_name = lin.split(',')[4]
        if spp_name in taxon_name_to_label:
            label = taxon_name_to_label[spp_name]
            ncbi_id = lin.split(',')[9]
            if label in label_to_ncbi:
                if not ncbi_id == label_to_ncbi[label].strip('ncbi:'):
                    print("{}, {}, {}".format(label, ncbi_id, label_to_ncbi[label]))
        

Abeomelomys_sevia_MURIDAE_RODENTIA, 491889, ncbi:491870
Abrocoma_bennettii_ABROCOMIDAE_RODENTIA, 126352, ncbi:108855
Abrocoma_boliviensis_ABROCOMIDAE_RODENTIA, 0, ncbi:1568925
Abrothrix_andinus_CRICETIDAE_RODENTIA, 1392492, ncbi:89096
Abrothrix_andinus_CRICETIDAE_RODENTIA, 10071, ncbi:89096
Abrothrix_jelskii_CRICETIDAE_RODENTIA, 1392495, ncbi:241142
Abrothrix_longipilis_CRICETIDAE_RODENTIA, 1392504, ncbi:29094
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 70165, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 70166, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 70167, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 156370, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 156197, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 70168, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 156199, ncbi:29098
Abrothrix_olivaceus_CRICETIDAE_RODENTIA, 156200, ncbi:29098
Acomys_cahirinus_MURIDAE_RODENTIA, 645539, ncbi:10068
Aepyceros_melampus_BOVIDAE_CETARTIODACTYLA, 141462, ncbi

Lagidium_ahuacaense_CHINCHILLIDAE_RODENTIA, 439226, 
Microgale_grandidieri_TENRECIDAE_AFROSORICIDA, 0, ncbi:1833855
Murina_annamitica_VESPERTILIONIDAE_CHIROPTERA, 867848, 
Murina_chrysochaetes_VESPERTILIONIDAE_CHIROPTERA, 867851, 
Murina_fionae_VESPERTILIONIDAE_CHIROPTERA, 867847, 
Murina_loreliae_VESPERTILIONIDAE_CHIROPTERA, 1046851, -
Murina_shuipuensis_VESPERTILIONIDAE_CHIROPTERA, 1046852, 
Murina_walstoni_VESPERTILIONIDAE_CHIROPTERA, 867846, 
Musseromys_gulantang_MURIDAE_RODENTIA, 685449, 
Peropteryx_pallidoptera_EMBALLONURIDAE_CHIROPTERA, 463810, 
Petaurista_grandis_SCIURIDAE_RODENTIA, 209424, -
Petaurista_lena_SCIURIDAE_RODENTIA, 209427, -
Petaurista_marica_SCIURIDAE_RODENTIA, 0, -
Platyrrhinus_angustirostris_PHYLLOSTOMIDAE_CHIROPTERA, 679017, -
Platyrrhinus_fusciventris_PHYLLOSTOMIDAE_CHIROPTERA, 679018, -
Rhinolophus_cohenae_RHINOLOPHIDAE_CHIROPTERA, 0, 
Rhinolophus_mabuensis_RHINOLOPHIDAE_CHIROPTERA, 0, 
Rhinolophus_mossambicus_RHINOLOPHIDAE_CHIROPTERA, 0, 
Rhinolophus_smither

In [63]:
taxon_name_to_label['Ornithorhynchus_anatinus']

'Ornithorhynchus_anatinus_ORNITHORHYNCHIDAE_MONOTREMATA'

In [96]:
fi = open('mammal/rojas_bats.txt').readlines()

In [97]:
old = []
new = []
for lin in fi:
    if lin.strip() in spp_set:
        old.append(lin.strip())
    else:
        new.append(lin.strip())

In [98]:
len(new)

38

In [99]:
len(old)

157

In [100]:
new

['Artibeus anderseni',
 'Artibeus aztecus',
 'Artibeus cinereus',
 'Artibeus glaucus',
 'Artibeus gnomus',
 'Artibeus incomitatus',
 'Artibeus intermedius',
 'Artibeus phaeotis',
 'Artibeus schwartzi',
 'Artibeus toltecus',
 'Brachyphylla nana nana',
 'Brachyphylla nana pumila',
 'Gardnerycteris crenulatum',
 'Lonchophylla dekeyseri',
 'Lophostoma silvicolum centralis',
 'Lophostoma silvicolum laephotis',
 'Macrotus waterhousii mexicanus',
 'Macrotus waterhousii waterhousii',
 'Micronycteris sp.',
 'Phyllonycteris poeyi obtusa',
 'Phyllonycteris poeyi poeyi',
 'Platyrrhinus incarum',
 'Platyrrhinus nigellus',
 'Pteronotus davyi davyi',
 'Pteronotus davyi fulvus',
 'Pteronotus parnellii parnellii',
 'Pteronotus parnellii portoricensis',
 'Pteronotus parnellii pusillus',
 'Pteronotus parnellii rubiginosus 1',
 'Pteronotus parnellii rubiginosus 2',
 'Pteronotus parnellii rubiginosus 3',
 'Pteronotus personatus personatus',
 'Pteronotus personatus psilotis',
 'Sturnira sp.',
 'Vampyriscus 